In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Paclitaxel Dose Optimization - Data Loading & EDA\n",
    "\n",
    "This notebook covers:\n",
    "1. Data loading from Excel file\n",
    "2. Exploratory Data Analysis (EDA)\n",
    "3. Data quality assessment\n",
    "4. Initial visualizations"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from scipy import stats\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Set style\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette('husl')\n",
    "\n",
    "print('Libraries imported successfully!')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load data from Excel file\n",
    "file_path = '../data/raw/Book1_1.xlsx'  # Update path as needed\n",
    "\n",
    "try:\n",
    "    # Load data from the correct sheet\n",
    "    df = pd.read_excel(file_path, sheet_name='DOZ X CANLILIK')\n",
    "    print('Data loaded successfully!')\n",
    "    print(f'Shape: {df.shape}')\n",
    "    print(f'Columns: {df.columns.tolist()}')\n",
    "except FileNotFoundError:\n",
    "    print('Excel file not found. Please place Book1_1.xlsx in the ../data/raw/ directory')\n",
    "except Exception as e:\n",
    "    print(f'Error loading data: {e}')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Display basic information about the dataset\n",
    "print('Dataset Overview:')\n",
    "print(f'Total samples: {len(df)}')\n",
    "print(f'Unique drugs: {df[\"DRUG_NAME\"].nunique()}')\n",
    "print(f'Unique cell lines: {df[\"ARXSPAN_ID\"].nunique()}')\n",
    "print(f'Dose range: {df[\"dose\"].min():.6f} - {df[\"dose\"].max():.6f}')\n",
    "print(f'Viability range: {df[\"viability\"].min():.3f} - {df[\"viability\"].max():.3f}')\n",
    "\n",
    "print('\\nData Info:')\n",
    "df.info()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check for missing values\n",
    "print('Missing Values:')\n",
    "missing_values = df.isnull().sum()\n",
    "print(missing_values[missing_values > 0])\n",
    "\n",
    "# Drug distribution\n",
    "print('\\nDrug Distribution:')\n",
    "print(df['DRUG_NAME'].value_counts())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Filter for Paclitaxel data (our focus)\n",
    "df_paclitaxel = df[df['DRUG_NAME'] == 'PACLITAXEL'].copy()\n",
    "print(f'Paclitaxel data shape: {df_paclitaxel.shape}')\n",
    "print(f'Unique cell lines for Paclitaxel: {df_paclitaxel[\"ARXSPAN_ID\"].nunique()}')\n",
    "\n",
    "# Cell line distribution for Paclitaxel\n",
    "print('\\nCell Line Distribution (Paclitaxel):')\n",
    "cell_counts = df_paclitaxel['ARXSPAN_ID'].value_counts()\n",
    "print('Top 10 cell lines:')\n",
    "print(cell_counts.head(10))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Statistical summary\n",
    "print('Statistical Summary:')\n",
    "print(df_paclitaxel[['dose', 'viability']].describe())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create comprehensive visualizations\n",
    "fig, axes = plt.subplots(3, 3, figsize=(18, 15))\n",
    "\n",
    "# 1. Dose distribution\n",
    "axes[0,0].hist(df_paclitaxel['dose'], bins=30, alpha=0.7, color='skyblue')\n",
    "axes[0,0].set_title('Dose Distribution')\n",
    "axes[0,0].set_xlabel('Dose (µM)')\n",
    "axes[0,0].set_ylabel('Frequency')\n",
    "axes[0,0].set_xscale('log')\n",
    "\n",
    "# 2. Viability distribution\n",
    "axes[0,1].hist(df_paclitaxel['viability'], bins=30, alpha=0.7, color='lightgreen')\n",
    "axes[0,1].set_title('Viability Distribution')\n",
    "axes[0,1].set_xlabel('Viability')\n",
    "axes[0,1].set_ylabel('Frequency')\n",
    "\n",
    "# 3. Dose vs Viability scatter\n",
    "axes[0,2].scatter(df_paclitaxel['dose'], df_paclitaxel['viability'], alpha=0.3, s=10)\n",
    "axes[0,2].set_title('Dose vs Viability')\n",
    "axes[0,2].set_xlabel('Dose (µM)')\n",
    "axes[0,2].set_ylabel('Viability')\n",
    "axes[0,2].set_xscale('log')\n",
    "\n",
    "# 4. Cell line sample counts\n",
    "top_10_cells = cell_counts.head(10)\n",
    "axes[1,0].bar(range(len(top_10_cells)), top_10_cells.values)\n",
    "axes[1,0].set_title('Top 10 Cell Lines (Sample Count)')\n",
    "axes[1,0].set_xlabel('Cell Line')\n",
    "axes[1,0].set_ylabel('Sample Count')\n",
    "axes[1,0].tick_params(axis='x', rotation=45)\n",
    "\n",
    "# 5. Dose-response curves for top cell lines\n",
    "top_5_cells = cell_counts.head(5).index\n",
    "for i, cell_line in enumerate(top_5_cells):\n",
    "    cell_data = df_paclitaxel[df_paclitaxel['ARXSPAN_ID'] == cell_line]\n",
    "    cell_data_sorted = cell_data.sort_values('dose')\n",
    "    axes[1,1].plot(cell_data_sorted['dose'], cell_data_sorted['viability'],\n",
    "                   marker='o', markersize=3, label=cell_line)\n",
    "axes[1,1].set_title('Dose-Response Curves (Top 5 Cell Lines)')\n",
    "axes[1,1].set_xlabel('Dose (µM)')\n",
    "axes[1,1].set_ylabel('Viability')\n",
    "axes[1,1].set_xscale('log')\n",
    "axes[1,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')\n",
    "\n",
    "# 6. Viability by dose bins\n",
    "dose_bins = pd.cut(df_paclitaxel['dose'], bins=10)\n",
    "viability_by_dose = df_paclitaxel.groupby(dose_bins)['viability'].mean()\n",
    "axes[1,2].plot(range(len(viability_by_dose)), viability_by_dose.values, 'o-')\n",
    "axes[1,2].set_title('Average Viability by Dose Bins')\n",
    "axes[1,2].set_xlabel('Dose Bins (Low to High)')\n",
    "axes[1,2].set_ylabel('Average Viability')\n",
    "\n",
    "# 7. Log dose vs viability\n",
    "df_paclitaxel['log_dose'] = np.log10(df_paclitaxel['dose'])\n",
    "axes[2,0].scatter(df_paclitaxel['log_dose'], df_paclitaxel['viability'], alpha=0.3, s=10)\n",
    "axes[2,0].set_title('Log Dose vs Viability')\n",
    "axes[2,0].set_xlabel('Log Dose')\n",
    "axes[2,0].set_ylabel('Viability')\n",
    "\n",
    "# 8. Viability distribution by dose quartiles\n",
    "dose_quartiles = pd.qcut(df_paclitaxel['dose'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])\n",
    "df_paclitaxel['dose_quartile'] = dose_quartiles\n",
    "quartile_data = []\n",
    "for q in ['Q1', 'Q2', 'Q3', 'Q4']:\n",
    "    quartile_data.append(df_paclitaxel[df_paclitaxel['dose_quartile'] == q]['viability'].values)\n",
    "axes[2,1].boxplot(quartile_data, labels=['Q1', 'Q2', 'Q3', 'Q4'])\n",
    "axes[2,1].set_title('Viability Distribution by Dose Quartiles')\n",
    "axes[2,1].set_xlabel('Dose Quartiles')\n",
    "axes[2,1].set_ylabel('Viability')\n",
    "\n",
    "# 9. Correlation heatmap\n",
    "numeric_cols = ['dose', 'viability', 'log_dose']\n",
    "corr_matrix = df_paclitaxel[numeric_cols].corr()\n",
    "im = axes[2,2].imshow(corr_matrix, cmap='coolwarm', aspect='auto')\n",
    "axes[2,2].set_title('Correlation Matrix')\n",
    "axes[2,2].set_xticks(range(len(numeric_cols)))\n",
    "axes[2,2].set_yticks(range(len(numeric_cols)))\n",
    "axes[2,2].set_xticklabels(numeric_cols)\n",
    "axes[2,2].set_yticklabels(numeric_cols)\n",
    "plt.colorbar(im, ax=axes[2,2])\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Outlier detection\n",
    "print('Outlier Detection:')\n",
    "Q1 = df_paclitaxel['viability'].quantile(0.25)\n",
    "Q3 = df_paclitaxel['viability'].quantile(0.75)\n",
    "IQR = Q3 - Q1\n",
    "lower_bound = Q1 - 1.5 * IQR\n",
    "upper_bound = Q3 + 1.5 * IQR\n",
    "outliers = df_paclitaxel[(df_paclitaxel['viability'] < lower_bound) |\n",
    "                         (df_paclitaxel['viability'] > upper_bound)]\n",
    "print(f'Number of outliers: {len(outliers)}')\n",
    "print(f'Outlier percentage: {len(outliers)/len(df_paclitaxel)*100:.2f}%')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save clean data for next steps\n",
    "output_path = '../data/processed/paclitaxel_clean.csv'\n",
    "df_paclitaxel.to_csv(output_path, index=False)\n",
    "print(f'Clean data saved to: {output_path}')\n",
    "print(f'Final dataset shape: {df_paclitaxel.shape}')\n",
    "print('EDA completed! Data is ready for feature engineering.')"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}